In [34]:
import random
import seaborn as sns
import numpy as xnp
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import time
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np

from sklearn.metrics import adjusted_rand_score
#from sklearn.cluster import AgglomerativeClustering

#library for euclidean distance calculation
from scipy.spatial.distance import pdist, squareform

#library for Entropy calculation
from scipy.stats import entropy

#library for Calinski-Harabasz score calculation
from sklearn.metrics import calinski_harabasz_score

#library for davies_bouldin score calculation
from sklearn.metrics import davies_bouldin_score

#library for table style
from IPython.display import display, HTML

from gap_statistic import OptimalK

from sklearn.metrics import pairwise_distances

#python font increase library
from termcolor import colored

#to save output
from nbconvert import HTMLExporter
import nbformat
from nbconvert.preprocessors import ExecutePreprocessor


#function for dunn_index score calculation
def dunn_index(X, labels):
    unique_labels = np.unique(labels)
    min_intercluster_distance = np.inf
    max_intracluster_diameter = -np.inf

    for i in range(len(unique_labels)):
        for j in range(i + 1, len(unique_labels)):
            cluster_i_points = X[labels == unique_labels[i]]
            cluster_j_points = X[labels == unique_labels[j]]
            
            # Calculate the minimum distance between clusters
            intercluster_distance = np.min(pairwise_distances(cluster_i_points, cluster_j_points))
            min_intercluster_distance = min(min_intercluster_distance, intercluster_distance)

        # Calculate the maximum intra-cluster diameter
        intracluster_diameter = np.max(pairwise_distances(X[labels == unique_labels[i]]))
        max_intracluster_diameter = max(max_intracluster_diameter, intracluster_diameter)

        if max_intracluster_diameter == 0:
            return 0  # Return zero or any other appropriate value
        else:
            return min_intercluster_distance / max_intracluster_diameter

#set plot style with default seaborn style
sns.set()

silhouette_scores = []  
entropy_scores = []
calinski_harabasz_scores = []
davies_bouldin_scores = []
dunn_index_scores = [] 
gap_scores = []

tempo_di_calcolo_silhouette = 0
tempo_di_calcolo_entropy = 0
tempo_di_calcolo_calinski_harabasz = 0
tempo_di_calcolo_davies_bouldin = 0
tempo_di_calcolo_dunn_index = 0
tempo_di_calcolo_gap = 0

lista_tempo_di_calcolo_silhouette = [] 
lista_tempo_di_calcolo_entropy = [] 
lista_tempo_di_calcolo_calinski_harabasz = [] 
lista_tempo_di_calcolo_davies_bouldin = [] 
lista_tempo_di_calcolo_dunn_index = [] 
lista_tempo_di_calcolo_gap = [] 


#metric trend calculation
silhouette_final_trend = []      
entropy_final_trend = []
calinski_harabasz_final_trend = []
davies_bouldin_final_trend = []
dunn_index_final_trend = [] 
gap_final_trend = []

# List of dataset filenames
dataset_files = [
    'toy_dataset_grouped.csv',
    'Takashi2019_diabetes_type1_dataset_preprocessed.csv',
    'journal.pone.0175818_S1Dataset_Spain_cardiac_arrest_EDITED.csv',
    'journal.pone.0158570_S2File_depression_heart_failure.csv',
    'journal.pone.0148699_S1_Text_Sepsis_SIRS_EDITED.csv',
    'neuroblastoma.csv'
]

y=0
k = 0
c=1
x=0
for dataset_file in dataset_files:
    titolo = colored("Processing dataset:" + str(dataset_file), attrs=['bold', 'reverse'])
    print(titolo+"\n\n")
    # Read the dataset
    df = pd.read_csv(dataset_file)
    
    # Shuffle the dataset
    row_indices = list(range(len(df)))
    new_order = random.sample(row_indices, len(row_indices))
    df = df.iloc[new_order]
    
    # Normalize each column
    df = (df - df.min()) / (df.max() - df.min())

    # Calculate the maximum possible Euclidean distance
    arr = np.zeros((2, df.shape[1]))
    arr[1:] = 1
    max_euclidean_distances= pdist(arr, metric='euclidean')

    # Thresholds for clustering
    soglie = [0.2, 0.33]


    for soglia in soglie:
    
        soglia_percentuale = str(soglia * 100) + "%"
        soglia = soglia * max_euclidean_distances
    
        # Calcolo delle distanze euclidee
        euclidean_distances= pdist(df.values, metric='euclidean')
        distance_matrix = squareform(euclidean_distances)
        
        cluster_labels_euclidean = []
        
        # Initialize an array to store cluster labels
        cluster_labels = [-1] * len(df)
        current_label = 0
        
        # Assign cluster labels based on the threshold
        #parte da sistemare
        for i in range(len(df)):
            if cluster_labels[i] == -1:
                cluster_labels[i] = current_label
                for j in range(i+1, len(df)):
                    if distance_matrix[i, j] < soglia and cluster_labels[j] == -1:
                        cluster_labels[j] = current_label
                current_label += 1
                if current_label == 2:
                    break
    
        count_0 = cluster_labels.count(0)
        count_1 = cluster_labels.count(1)
    
        #create a mask that contain only the rows with a cluster_labels valued != -1
        mask = np.array(cluster_labels) != -1
        #apply the mask to generate the matrix with only the rows that i want
        df_selected = df[mask]
    
        mask = np.array(cluster_labels) != -1
        cluster_labels_selected = np.array(cluster_labels)[mask]
        
        testo_grande = colored("result with threshold of "+ str(soglia_percentuale), attrs=['bold', 'reverse'])
        print(testo_grande+"\n")
        
        # Print cluster labels
        print(f"Clusters with threshold {soglia_percentuale}: {cluster_labels}")
        print(f"Clusters with Euclidean distance : {cluster_labels_selected}")
        
        
        # Clustering calculation with k-means
        random_state = 45
        kmeans = KMeans(n_clusters=2,random_state=random_state)
        kmeans_clusters = kmeans.fit_predict(df_selected)
        l= kmeans.labels_
        
        #count how many elements there are for each cluster
        cluster_counts = np.bincount(l)
        print(f"Clusters with K-means : {kmeans_clusters}")

        trovato = False
        if len(l) > 2: 

            trovato = True

            
            # Silhouette calculation
            start_time = time.time()
            silhouette_avg = silhouette_score(df_selected, l)
            silhouette_scores.append(silhouette_avg)
            end_time = time.time()
            tempo_di_calcolo = end_time - start_time
            lista_tempo_di_calcolo_silhouette.append(tempo_di_calcolo)
            
            # Entropy calculation (complementary Entropy)
            start_time = time.time()
            entropy_val = entropy(np.bincount(l) / len(l), base=2)
            entropy_val = abs(1-entropy_val)
            entropy_scores.append(entropy_val)
            end_time = time.time()
            tempo_di_calcolo = end_time - start_time
            lista_tempo_di_calcolo_entropy.append(tempo_di_calcolo)
        
            # Calinski-Harabasz index calculation
            start_time = time.time()
            calinski_harabasz_index = calinski_harabasz_score(df_selected, l)
            calinski_harabasz_scores.append(calinski_harabasz_index)
            end_time = time.time()
            tempo_di_calcolo = end_time - start_time
            lista_tempo_di_calcolo_calinski_harabasz.append(tempo_di_calcolo)
        
            # Davies-Bouldin calculation (reciprocal function)
            start_time = time.time()
            davies_bouldin_avg = davies_bouldin_score(df_selected, l)
            davies_bouldin_avg = 1 / davies_bouldin_avg
            davies_bouldin_scores.append(davies_bouldin_avg)
            end_time = time.time()
            tempo_di_calcolo = end_time - start_time
            lista_tempo_di_calcolo_davies_bouldin.append(tempo_di_calcolo)
        
            # Dunn index calculation
            start_time = time.time()
            dunn_score = dunn_index(df_selected, cluster_labels_selected)
            dunn_index_scores.append(dunn_score)
            end_time = time.time()
            tempo_di_calcolo = end_time - start_time
            lista_tempo_di_calcolo_dunn_index.append(tempo_di_calcolo)
        
            # Gap Statistic calculation
            start_time = time.time()
            optimal_k = OptimalK(parallel_backend='joblib')
            n_clusters = optimal_k(df_selected, cluster_array=np.arange(1, 3))
            gap_df = optimal_k.gap_df
            optimal_gap_statistic = gap_df.loc[gap_df.gap_value.idxmax(), 'gap_value']
            gap_scores.append(optimal_gap_statistic)
            end_time = time.time()
            tempo_di_calcolo = end_time - start_time
            lista_tempo_di_calcolo_gap.append(tempo_di_calcolo)
    
            print("")
            print("Metrics calculated on the labels obtained from the Euclidean distance:")
            print("Silhouette Score:", silhouette_scores[y])
            print("Complementary Entropy Score:", entropy_scores[y])
            print("Calinski-Harabasz Score:", calinski_harabasz_scores[y])
            print("Reciprocal Davies-Bouldin Score:", davies_bouldin_scores[y])
            print("Dunn index Score:", dunn_index_scores[y])
            print("Gap Statistic Score:", gap_scores[y])
            print("")
        
        # application of Adjusted Rand Index
        ari = adjusted_rand_score(cluster_labels_selected, kmeans_clusters)
        print("\nTo compare the results obtained with the Euclidean distance and those obtained with k-means I use the range adjusted Rand index")
        print("range adjusted Rand index: [-1, 1]")
        print(f"Threshold: {soglia_percentuale}, adjusted Rand index: {ari}")

        if trovato == True:
            print("\nConsistency between Silhouette and Adjusted Rand Index:")
            rapport_Silhouette_ARI = abs(ari - silhouette_scores[y])
            
            if rapport_Silhouette_ARI < 0.333:
                print("The Silhouette and the Adjusted Rand Index are coherent")
            elif rapport_Silhouette_ARI > 0.333 and rapport_Silhouette_ARI < 1:
                print("The Silhouette and the Adjusted Rand Index are sufficiently coherent")
            else:
                print("The Silhouette and the Adjusted Rand Index are inconsistent")

            #save output in a file
            with open("/Users/andreaspagnolo/Desktop/uni/stage/result/result_real_data/output.txt", "w") as file:
                file.write("result with threshold of "+ str(soglia_percentuale) + "\n")
                file.write(f"Clusters with threshold {soglia_percentuale}: {cluster_labels}\n")
                file.write(f"Clusters with Euclidean distance : {cluster_labels_selected}\n")
                file.write(f"Clusters with K-means : {kmeans_clusters}\n\n")
                file.write("Metrics calculated on the labels obtained from the Euclidean distance:\n")
                file.write(f"Silhouette Score: {silhouette_scores[y]}\n")
                file.write(f"Complementary Entropy Score: {entropy_scores[y]}\n")
                file.write(f"Calinski-Harabasz Score: {calinski_harabasz_scores[y]}\n")
                file.write(f"Reciprocal Davies-Bouldin Score: {davies_bouldin_scores[y]}\n")
                file.write(f"Dunn index Score: {dunn_index_scores[y]}\n")
                file.write(f"Gap Statistic Score: {gap_scores[y]}\n\n")
                file.write("To compare the results obtained with the Euclidean distance and those obtained with k-means I use the range adjusted Rand index\n")
                file.write("range adjusted Rand index: [-1, 1]\n")
                file.write(f"Threshold: {soglia_percentuale}, adjusted Rand index: {ari}\n")
                file.write("\nConsistency between Silhouette and Adjusted Rand Index:")
                if rapport_Silhouette_ARI < 0.333:
                    file.write("The Silhouette and the Adjusted Rand Index are coherent")
                elif rapport_Silhouette_ARI > 0.333 and rapport_Silhouette_ARI < 1:
                    file.write("The Silhouette and the Adjusted Rand Index are sufficiently coherent")
                else:
                    file.write("The Silhouette and the Adjusted Rand Index are inconsistent")
                file.write("\nConsistency between Silhouette and Adjusted Rand Index:")
            
        if trovato == True:
            y+=1
       
    
        #table with Euclidean distance clustering values
        table_df = pd.DataFrame(columns=["cluster", "cluster elements", "%"])
        
        dati_tabella = [
            ["1st cluster", str(count_0) + " patients out of " + str(df.shape[0]) , count_0  / df.shape[0] * 100],
            ["2nd cluster", str(count_1) + " patients out of " + str(df.shape[0]), count_1  / df.shape[0] * 100],
            ["no cluster",str(df.shape[0]-(count_0+count_1))+ " patients out of " + str(df.shape[0]), (df.shape[0]-(count_0+count_1)) / df.shape[0] * 100],
        ]
        
        for row_data in dati_tabella:
            table_df.loc[len(table_df)] = row_data
            
        table_df.style \
          .format(precision=3, thousands=".", decimal=",").set_properties(**{'text-align': 'left'})
        
        html_table = table_df.to_html(index=False, classes="table table-striped", border=0)
        html_table = html_table.replace('<table', '<table style="width:100%;" text-align:center;"')
        html_table = html_table.replace('<td', '<td style="text-align:center;"').replace('<th', '<th style="text-align:center;"')
    
        print("\n")
        html_table = "<caption>Table with Euclidean distance clustering values:</caption>" + html_table
    
        
        table_df.to_csv("/Users/andreaspagnolo/Desktop/uni/stage/result/result_real_data/euclidean_distance_result" + ".csv")
        
        #print table
        display(HTML(html_table))
    
        #print % of correct classification
        var1 = round((( count_0+count_1 ) / df.shape[0]) * 100 ,3)
        var2 = round(((df.shape[0]-(count_0+count_1))/df.shape[0])*100 ,3)
        print(str(var1)  + "% of the elements were correctly classified into the two clusters. The remaining " + str(var2) + "% of items were not classified")
    
    
        
        #Table with K-means clustering values
        table_df_2 = pd.DataFrame(columns=["cluster", "cluster elements", "%"])
        
        dati_tabella2 = [
            ["1st cluster", str(cluster_counts[0]) + " patients out of " + str(df.shape[0]) , cluster_counts[0]  / df.shape[0] * 100],
            ["2nd cluster", str(cluster_counts[1]) + " patients out of " + str(df.shape[0]), cluster_counts[1]  / df.shape[0] * 100],
            ["no cluster", str(df.shape[0] - (cluster_counts[0] + cluster_counts[1])) + " patients out of " + str(df.shape[0]), (df.shape[0] - (cluster_counts[0] + cluster_counts[1])) / df.shape[0] * 100],
        ]
    
        
        for row_data in dati_tabella2:
            table_df_2.loc[len(table_df_2)] = row_data
            
        table_df_2.style \
          .format(precision=3, thousands=".", decimal=",").set_properties(**{'text-align': 'left'})
        
        html_table2 = table_df_2.to_html(index=False, classes="table table-striped", border=0)
        html_table2 = html_table2.replace('<table', '<table style="width:100%;" text-align:center;"')
        html_table2 = html_table2.replace('<td', '<td style="text-align:center;"').replace('<th', '<th style="text-align:center;"')
    
        print("\n")
        html_table2 = "<caption>Table with K-means clustering values:</caption>" + html_table2   
    
        table_df_2.to_csv("/Users/andreaspagnolo/Desktop/uni/stage/result/result_real_data/kmeans_result" + ".csv")
        
        #print table
        display(HTML(html_table2))
    
        

    #metric trend calculation
    silhouette_trend = []      
    entropy_trend = []
    calinski_harabasz_trend = []
    davies_bouldin_trend = []
    dunn_index_trend = [] 
    gap_trend = []
    
    silhouette_range = 2
    entropy_range = 1
    davies_bouldin_range = abs(davies_bouldin_scores[k]-davies_bouldin_scores[-1])
    dunn_index_range = abs(dunn_index_scores[k]-dunn_index_scores[-1])
    calinski_harabasz_range = abs(calinski_harabasz_scores[k]-calinski_harabasz_scores[-1])
    gap_range = abs(gap_scores[k]-gap_scores[-1])
    k +=2
    
    trend = [] 
    def calculate_trend(current_value, previous_value, metric_scores,metric_range, threshold=5):
        cambiamento_minimo = metric_range * 0.01
        
        #final_change = metric_scores[0]-metric_scores[-1]
        if previous_value != 0:
            change = ((current_value - previous_value)/previous_value) * 100
        else:
            change = 0
        if change > threshold:
            return "Wrong" #incresing
        else:
            return "Correct" #decresing

    i=y-1
    irregular=0
    for i in range(i, len(silhouette_scores)):
        trend_per_iteration = calculate_trend(silhouette_scores[i], silhouette_scores[i-1], silhouette_scores,silhouette_range)
        silhouette_trend.append(trend_per_iteration)
        for i in range (len(silhouette_trend)):
            if i > 0 and silhouette_trend[i] != silhouette_trend[i-1]:
                irregular = irregular + 1
    if irregular >= 1:
        trend.append("Wrong")
    else:
        trend.append(silhouette_trend[-1])
    
    i=y-1
    irregular=0
    for i in range(i, len(entropy_scores)):
        trend_per_iteration = calculate_trend(entropy_scores[i], entropy_scores[i-1], entropy_scores,entropy_range)
        entropy_trend.append(trend_per_iteration)
        for i in range (len(entropy_trend)):
            if i > 0 and entropy_trend[i] != entropy_trend[i-1]:
                irregular = irregular + 1
    if irregular >= 1:
        trend.append("Wrong")
    else:
        trend.append(entropy_trend[-1])
    
    i=y-1
    irregular=0
    for i in range(i, len(davies_bouldin_scores)):
        trend_per_iteration = calculate_trend(davies_bouldin_scores[i], davies_bouldin_scores[i-1], davies_bouldin_scores,davies_bouldin_range)
        davies_bouldin_trend.append(trend_per_iteration)
        for i in range (len(davies_bouldin_trend)):
            if i > 0 and davies_bouldin_trend[i] != davies_bouldin_trend[i-1]:
                irregular = irregular + 1
    if irregular >= 1:
        trend.append("Wrong")
    else:
        trend.append(davies_bouldin_trend[-1])
    
    i=y-1
    irregular=0
    for i in range(i, len(dunn_index_scores)):
        trend_per_iteration = calculate_trend(dunn_index_scores[i], dunn_index_scores[i-1], dunn_index_scores,dunn_index_range)
        dunn_index_trend.append(trend_per_iteration)
        for i in range (len(dunn_index_trend)):
            if i > 0 and dunn_index_trend[i] != dunn_index_trend[i-1]:
                irregular = irregular + 1
    if irregular >= 1:
        trend.append("Wrong")
    else:
        trend.append(dunn_index_trend[-1])
    
    i=y-1
    irregular=0
    for i in range(i, len(calinski_harabasz_scores)):
        trend_per_iteration = calculate_trend(calinski_harabasz_scores[i], calinski_harabasz_scores[i-1], calinski_harabasz_scores,calinski_harabasz_range)
        calinski_harabasz_trend.append(trend_per_iteration)
        for i in range (len(calinski_harabasz_trend)):
            if i > 0 and calinski_harabasz_trend[i] != calinski_harabasz_trend[i-1]:
                irregular = irregular + 1
    if irregular >= 1:
        trend.append("Wrong")
    else:
        trend.append(calinski_harabasz_trend[-1])
    
    i=y-1
    irregular=0
    for i in range(i, len(gap_scores)):
        trend_per_iteration = calculate_trend(gap_scores[i], gap_scores[i-1], gap_scores,gap_range)
        gap_trend.append(trend_per_iteration)
        for i in range (len(gap_trend)):
            if i > 0 and gap_trend[i] != gap_trend[i-1]:
                irregular = irregular + 1
    if irregular >= 1:
        trend.append("Wrong")
    else:
        trend.append(gap_trend[-1])

    silhouette_final_trend.append(trend[0])     
    entropy_final_trend.append(trend[1])    
    davies_bouldin_final_trend.append(trend[2])
    dunn_index_final_trend.append(trend[3])    
    calinski_harabasz_final_trend.append(trend[4])   
    gap_final_trend.append(trend[5])    


    #table with result for a single dataset
    print("\n\n")
    table_df3 = pd.DataFrame(columns=["metric", "result", "time"])
    
    #time calculation for single dataset
    if len(lista_tempo_di_calcolo_silhouette) == x+2:
        tempo1 = (lista_tempo_di_calcolo_silhouette[x] + lista_tempo_di_calcolo_silhouette[x+1]) / 2
        tempo2 = (lista_tempo_di_calcolo_entropy[x] + lista_tempo_di_calcolo_entropy[x+1]) / 2
        tempo3 = (lista_tempo_di_calcolo_calinski_harabasz[x] + lista_tempo_di_calcolo_calinski_harabasz[x+1]) / 2
        tempo4 = (lista_tempo_di_calcolo_davies_bouldin[x] + lista_tempo_di_calcolo_davies_bouldin[x+1]) / 2
        tempo5 = (lista_tempo_di_calcolo_dunn_index[x] + lista_tempo_di_calcolo_dunn_index[x+1]) / 2
        tempo6 = (lista_tempo_di_calcolo_gap[x] + lista_tempo_di_calcolo_gap[x+1]) / 2
        x +=2
    else:
        tempo1 = (lista_tempo_di_calcolo_silhouette[x])
        tempo2 = (lista_tempo_di_calcolo_entropy[x])
        tempo3 = (lista_tempo_di_calcolo_calinski_harabasz[x])
        tempo4 = (lista_tempo_di_calcolo_davies_bouldin[x])
        tempo5 = (lista_tempo_di_calcolo_dunn_index[x])
        tempo6 = (lista_tempo_di_calcolo_gap[x])
        x +=1
        
    dati_tabella3 = [
        ["Silhouette", trend[0], str(round (tempo1 *1000,3)) + " ms"],
        ["Complementary Entropy", trend[1], str(round (tempo2 *1000,3)) + " ms"],
        ["Reciprocal Davies-Bouldin", trend[2], str(round (tempo3*1000,3)) + " ms"],
        ["Dunn index", trend[3], str(round (tempo4*1000,3)) + " ms"],
        ["Calinski-Harabasz", trend[4], str(round (tempo5*1000,3)) + " ms"],
        ["Gap Statistic", trend[5], str(round (tempo6*1000,3)) + " ms"]
    ]
    
    for row_data in dati_tabella3:
        table_df3.loc[len(table_df3)] = row_data
    
    table_df3.style \
    .format(precision=3, thousands=".", decimal=",").set_properties(**{'text-align': 'left'})
    
    html_table3 = table_df3.to_html(index=False, classes="table table-striped", border=0)
    html_table3 = html_table3.replace('<table', '<table style="width:100%;" text-align:center;"')
    html_table3 = html_table3.replace('<td', '<td style="text-align:center;"').replace('<th', '<th style="text-align:center;"')
    
    print("\n")
    html_table3 = "<caption>Table with metric result:</caption>" + html_table3  
    
    table_df3.to_csv("/Users/andreaspagnolo/Desktop/uni/stage/result/result_real_data/metric_result" + ".csv")
    
    #print table
    display(HTML(html_table3))
    print("I expect the metrics to get worse as the threshold increases. All metrics are consistent with this worsening except entropy and Calinski-Harabasz")
    print("\n\n\n\n\n")
    c += 1



# count 'wrong' and 'correct' in silhouette_trend
wrong_count_silhouette = silhouette_final_trend.count('Wrong')
correct_count_silhouette = silhouette_final_trend.count('Correct')

# count 'wrong' and 'correct' in entropy_trend
wrong_count_entropy = entropy_final_trend.count('Wrong')
correct_count_entropy = entropy_final_trend.count('Correct')

# count 'wrong' and 'correct' in calinski_harabasz_trend
wrong_count_calinski_harabasz = calinski_harabasz_final_trend.count('Wrong')
correct_count_calinski_harabasz = calinski_harabasz_final_trend.count('Correct')

# count 'wrong' and 'correct' in davies_bouldin_trend
wrong_count_davies_bouldin = davies_bouldin_final_trend.count('Wrong')
correct_count_davies_bouldin = davies_bouldin_final_trend.count('Correct')

# count 'wrong' and 'correct' in dunn_index_trend
wrong_count_dunn_index = dunn_index_final_trend.count('Wrong')
correct_count_dunn_index = dunn_index_final_trend.count('Correct')

# count 'wrong' and 'correct' in gap_trend
wrong_count_gap = gap_final_trend.count('Wrong')
correct_count_gap = gap_final_trend.count('Correct')


#avarage time calculation for all dataset
tempo_di_calcolo_silhouette = (sum(lista_tempo_di_calcolo_silhouette)*1000) / len(lista_tempo_di_calcolo_silhouette)
tempo_di_calcolo_entropy = (sum(lista_tempo_di_calcolo_entropy)*1000) / len(lista_tempo_di_calcolo_entropy)
tempo_di_calcolo_davies_bouldin = (sum(lista_tempo_di_calcolo_davies_bouldin)*1000) / len(lista_tempo_di_calcolo_davies_bouldin)
tempo_di_calcolo_dunn_index = (sum(lista_tempo_di_calcolo_dunn_index)*1000) / len(lista_tempo_di_calcolo_dunn_index)
tempo_di_calcolo_calinski_harabasz = (sum(lista_tempo_di_calcolo_calinski_harabasz)*1000) / len(lista_tempo_di_calcolo_calinski_harabasz)
tempo_di_calcolo_gap = (sum(lista_tempo_di_calcolo_gap)*1000) / len(lista_tempo_di_calcolo_gap)

#table with result for all the datasets
table_df_4 = pd.DataFrame(columns=["metric", "correct", "wrong","%","avarage time" ])
dati_tabella_4 = [
    ["Silhouette", correct_count_silhouette, wrong_count_silhouette, f"{round((correct_count_silhouette / len(silhouette_final_trend)) * 100, 3)} %", f"{round(tempo_di_calcolo_silhouette,3)} ms"],
    ["Complementary Entropy", correct_count_entropy, wrong_count_entropy, f"{round((correct_count_entropy / len(entropy_final_trend)) * 100, 3)} %", f"{round(tempo_di_calcolo_entropy,3)} ms"],
    ["Reciprocal Davies-Bouldin", correct_count_davies_bouldin, wrong_count_davies_bouldin, f"{round((correct_count_davies_bouldin / len(davies_bouldin_final_trend)) * 100, 3)} %", f"{round(tempo_di_calcolo_davies_bouldin,3)} ms"],
    ["Dunn index", correct_count_dunn_index, wrong_count_dunn_index, f"{round((correct_count_dunn_index / len(dunn_index_final_trend)) * 100, 3)} %", f"{round(tempo_di_calcolo_dunn_index,3)} ms"],
    ["Calinski-Harabasz", correct_count_calinski_harabasz, wrong_count_calinski_harabasz, f"{round((correct_count_calinski_harabasz / len(calinski_harabasz_final_trend)) * 100, 3)} %", f"{round(tempo_di_calcolo_calinski_harabasz,3)} ms"],
    ["Gap Statistic", correct_count_gap, wrong_count_gap, f"{round((correct_count_gap / len(gap_final_trend)) * 100, 3)} %", f"{round(tempo_di_calcolo_gap,3)} ms"]
]

for row_data_4 in dati_tabella_4:
    table_df_4.loc[len(table_df_4)] = row_data_4

table_df_4 = table_df_4.sort_values(by="correct", ascending=False)

#table style
table_df_4.style \
  .format(precision=3, thousands=".", decimal=",").set_properties(**{'text-align': 'left'})

# Save HTML table to file
table_df_4.to_csv("/Users/andreaspagnolo/Desktop/uni/stage/result/result_real_data/final_report" + ".csv")

html_table_4 = table_df_4.to_html(index=False, classes="table table-striped", border=0)
html_table_4 = html_table_4.replace('<table', '<table style="width:100%;" text-align:center;"')
html_table_4 = html_table_4.replace('<td', '<td style="text-align:center;"').replace('<th', '<th style="text-align:center;"')

#print table
display(HTML(html_table_4))

Processing dataset:toy_dataset_grouped.csv


result with threshold of 20.0%

Clusters with threshold 20.0%: [0, 0, 1, 1, 1, 0, 0, 1, 1, 0]
Clusters with Euclidean distance : [0 0 1 1 1 0 0 1 1 0]
Clusters with K-means : [0 0 1 1 1 0 0 1 1 0]

Metrics calculated on the labels obtained from the Euclidean distance:
Silhouette Score: 0.9914807964068715
Complementary Entropy Score: 0.0
Calinski-Harabasz Score: 48464.58333333331
Reciprocal Davies-Bouldin Score: 90.78698187009488
Dunn index Score: 50.52062285180065
Gap Statistic Score: 6.980995531559505


To compare the results obtained with the Euclidean distance and those obtained with k-means I use the range adjusted Rand index
range adjusted Rand index: [-1, 1]
Threshold: 20.0%, adjusted Rand index: 1.0

Consistency between Silhouette and Adjusted Rand Index:
The Silhouette and the Adjusted Rand Index are coherent




cluster,cluster elements,%
1st cluster,5 patients out of 10,50.0
2nd cluster,5 patients out of 10,50.0
no cluster,0 patients out of 10,0.0


100.0% of the elements were correctly classified into the two clusters. The remaining 0.0% of items were not classified




cluster,cluster elements,%
1st cluster,5 patients out of 10,50.0
2nd cluster,5 patients out of 10,50.0
no cluster,0 patients out of 10,0.0


result with threshold of 33.0%

Clusters with threshold 33.0%: [0, 0, 1, 1, 1, 0, 0, 1, 1, 0]
Clusters with Euclidean distance : [0 0 1 1 1 0 0 1 1 0]
Clusters with K-means : [0 0 1 1 1 0 0 1 1 0]

Metrics calculated on the labels obtained from the Euclidean distance:
Silhouette Score: 0.9914807964068715
Complementary Entropy Score: 0.0
Calinski-Harabasz Score: 48464.58333333331
Reciprocal Davies-Bouldin Score: 90.78698187009488
Dunn index Score: 50.52062285180065
Gap Statistic Score: 6.851812000379764


To compare the results obtained with the Euclidean distance and those obtained with k-means I use the range adjusted Rand index
range adjusted Rand index: [-1, 1]
Threshold: 33.0%, adjusted Rand index: 1.0

Consistency between Silhouette and Adjusted Rand Index:
The Silhouette and the Adjusted Rand Index are coherent




...

29.586% of the elements were correctly classified into the two clusters. The remaining 70.414% of items were not classified




cluster,cluster elements,%
1st cluster,30 patients out of 169,17.751479
2nd cluster,20 patients out of 169,11.834320
no cluster,119 patients out of 169,70.414201


metric,result,time
Silhouette,Correct,3.721 ms
Complementary Entropy,Correct,0.756 ms
Reciprocal Davies-Bouldin,Correct,1.781 ms
Dunn index,Correct,2.407 ms
Calinski-Harabasz,Correct,1.206 ms
Gap Statistic,Correct,24.456 ms


I expect the metrics to get worse as the threshold increases. All metrics are consistent with this worsening except entropy and Calinski-Harabasz








metric,correct,wrong,%,avarage time
Silhouette,6,0,100.0 %,2.76 ms
Reciprocal Davies-Bouldin,6,0,100.0 %,1.36 ms
Dunn index,6,0,100.0 %,1.783 ms
Complementary Entropy,5,1,83.333 %,0.33 ms
Calinski-Harabasz,5,1,83.333 %,1.027 ms
Gap Statistic,5,1,83.333 %,20.866 ms
